In [ ]:
%pip install ultralytics
%pip install supervision
## Подключение гугл диска для сохранения модели
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Подсчет людей внутри зоны
# Смысл алгоритма: отметил на кадре 2 зоны, при попадании объекта внутрь зоны, запоминаем его на несколько кадров.
# Если объект перешел в другую зону, удаляю его из запомненых и увеличиваю счетчик входящих/выходящих
#
import cv2
import random
from ultralytics import YOLO
import supervision as sv
#from supervision.notebook.utils import show_frame_in_notebook
import numpy as np
from google.colab.patches import cv2_imshow
from IPython import display
from supervision import VideoInfo, VideoSink

%matplotlib inline
display.clear_output()

# Задаем координаты вершин полигона, который мы будем использовать для подсчета проходящих объектов
POLYGON_COORDS = []
#Определение по всему телу
#Camera1
POLYGON_COORDS.append([[391, 1], [429, 364], [937, 293], [838, 1]]) # 'Out'
POLYGON_COORDS.append([[398, 720], [432, 516], [998, 417], [1263, 720]]) # 'In'
#Camera2
#POLYGON_COORDS.append([[391, 1], [506, 627], [1059, 510], [838, 1]]) # 'Out'
#POLYGON_COORDS.append([[376, 1087], [381, 811], [1191, 670], [1713, 1099]]) # 'In'
#Camera3
#POLYGON_COORDS.append([[400, 1079], [588, 986], [1527, 670], [1800, 1079]]) # 'Out'
#POLYGON_COORDS.append([[400, 1], [518, 630], [1383, 382], [1000, 1]]) # 'In'


polygons=len(POLYGON_COORDS)
persons = [0 for _ in range(polygons)] # счетчик входящих/выходящих
deep = 500 # на скольки кадрах запоминаем людей
path='/content/gdrive/MyDrive/models/'
file_in='video/camera1_7in_4out.mp4'
file_out='camera1_7in_4out_s.mp4'
video_info = VideoInfo.from_video_path(video_path=path+file_in)
# Создаем экземпляр модели YOLO
model = YOLO("yolov8s.pt")
model = YOLO('/content/gdrive/MyDrive/models/yolov8s_modN.pt')  # load a custom model

def inPolygon(x, y, crd):  # Функция для определения лежит ли точка внутри многоугольника
    c = 0
    for i in range(len(crd)):
        if (((crd[i][1] <= y and y < crd[i - 1][1]) or (crd[i - 1][1] <= y and y < crd[i][1])) and
                (x > (crd[i - 1][0] - crd[i][0]) * (y - crd[i][1]) / (crd[i - 1][1] - crd[i][1]) + crd[i][
                    0])): c = 1 - c
    return c


def inLabel(p, lb, sl):  # выдает количество меток во всех полигонах кроме "sl"
    ret = 0
    for i in range(len(lb)):
        if i != sl:
            for j in range(len(lb[i])):
                ret += lb[i][j].count(p)
    return ret


def delFromLabel(p, lb):  # удаляет метку из массива
    for i in range(len(lb)):
        for j in range(len(lb[i])):
            for k in range(lb[i][j].count(p)):
                lb[i][j].remove(p)

def flatten_and_sort(lst):
    flattened_list = [item for sublist in lst for item in sublist]  # Разворачиваем список в одномерный список
    unique_sorted_list = sorted(set(flattened_list))  # Удаляем дубликаты и сортируем список
    return unique_sorted_list


# Создаем объекты для работы с подсчетом объектов и аннотированием полигона и прямоугольных рамок вокруг объектов

box_annotator = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5
)
box_annotator_c = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5,
    color=sv.Color.white()
)
kadr=0
labels_obj = [[[-1 for i in range(1)] for k in range(deep)] for l in
              range(polygons)]
# 1 количество полигонов
# 2 Глубина - сколько кадров храним данные
# 3 Метки объектов

#Запись результата
with VideoSink(target_path=path+file_out, video_info=video_info) as s:
    # Итерируемся по кадрам видео
    for result in model.track(source=path+file_in, show=False, stream=True, agnostic_nms=True,
                          classes=0, iou=0.7, conf=0.25, tracker="botsort.yaml"):  # tracker="bytetrack.yaml",tracker="botsort.yaml"
        kadr+=1
        # Получаем текущий кадр и результаты детекции
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Присваиваем каждому объекту ID, чтобы отслеживать его в последующих кадрах
        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

            labels = []
            for i in range(len(detections)):
                labels.append(f"{detections.tracker_id[i]} {model.model.names[detections.class_id[i]]} {detections.confidence[i]:0.2f}")



            # Аннотируем прямоугольными рамками объекты на кадре
            frame = box_annotator.annotate(
                scene=frame,
                detections=detections,
                labels=labels
            )
        #Умеьшаем рамки для детектирования в n раз
        n=100
        if result.boxes.id is not None:
            for i in range(len(detections.xyxy)):
                x1=detections.xyxy[i][0]
                y1=detections.xyxy[i][1]
                x2=detections.xyxy[i][2]
                y2=detections.xyxy[i][3]
                detections.xyxy[i][0]=x1+(x2-x1)/2-(x2-x1)/n
                detections.xyxy[i][1]=y1+(y2-y1)/2-(y2-y1)/n
                detections.xyxy[i][2]=x1+(x2-x1)/2+(x2-x1)/n
                detections.xyxy[i][3]=y1+(y2-y1)/2+(y2-y1)/n

        # Calculate
        for np1 in range(polygons):
            labels_obj[np1].pop(0)  # Удаляем верхний слой(самый старый)
            labels_obj[np1].append([])
        if detections.tracker_id is not None:
            for pl in range(polygons): #добавляем метки в полигоны
                cnt = len(detections.xyxy) #сколько объектов найдено на кадре
                for pers in range(cnt):
                    x = detections.xyxy[pers][0]
                    y = detections.xyxy[pers][1]
                    p = detections.tracker_id[pers]
                    if inPolygon(x, y, POLYGON_COORDS[pl]):
                        r = inLabel(p, labels_obj, pl)
                        if r:  # был ли объект в другом полигоне, если да, то удаляем его и прибавляем счетчик
                            delFromLabel(p, labels_obj)
                            persons[pl] += 1
                            print(p, end=' ')
                        else:
                            labels_obj[pl][deep - 1].append(p)  # добавляем метку объекта в полигон
        print('Out',flatten_and_sort(labels_obj[0]),' In',flatten_and_sort(labels_obj[1]), persons)

        # text
        frame = sv.draw.utils.draw_text(scene=frame, text='Out:'+str(persons[0]), text_anchor=sv.Point(x=150, y=200),text_color=sv.Color.white(),text_scale=1)
        frame = sv.draw.utils.draw_text(scene=frame, text=' In:'+str(persons[1]), text_anchor=sv.Point(x=150, y=250),text_color=sv.Color.white(),text_scale=1)
        # Рисуем прямоугольные рамки центров объектов на кадре
        frame = box_annotator_c.annotate(
            scene=frame,
            detections=detections,
            labels=None, skip_label=True
        )

        # Подсчитываем количество объектов, пересекающих полигон
        #zone1.trigger(detections=detections)
        # Подсчитываем количество объектов, пересекающих полигон
        #zone2.trigger(detections=detections)

        # Аннотируем полигон на кадре
        #frame = zone_annotator1.annotate(scene=frame)
        #frame = zone_annotator2.annotate(scene=frame)
        frame = sv.draw.utils.draw_polygon(scene=frame, polygon=np.array(POLYGON_COORDS[0]), color=sv.Color.white(),thickness=2)
        frame = sv.draw.utils.draw_polygon(scene=frame, polygon=np.array(POLYGON_COORDS[1]), color=sv.Color.white(),thickness=2)

        # Отображаем кадр
        s.write_frame(frame=frame)
        #sv.show_frame_in_notebook(frame, (16, 16))
        #cv2_imshow(frame)
        # запись видео в файл
        #out.write(frame)
        # Отображаем кадр
        #cv2.imshow("yolov8", frame)

        # Выходим из цикла, если пользователь нажал клавишу ESC
        if (cv2.waitKey(30) == 27):
            break
        #if kadr>5:
            #break
    #out.release()



In [ ]:
Результаты
	        Out/In      	Out/In   	Out/In
	        реальные    	yolo8m   	yolo8s обученная
Camera1			            36/33		  41/38
Camera2	    33/26		    32/29		  31/25	  Перекрытие
Camera3	    11/72		    11/61		  12/70	  Не достаточно кадров, не удачная линия пересечения
